# USGS Streamflow Interpolation Tool

Interactive Sink–Rise flow analysis using the USGS NWIS Instantaneous Values API.
Modify site date ranges below.

In [ ]:
from IPython.display import display
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets

def _normalize_dt(dt_in):
    dt = pd.to_datetime(dt_in)
    return dt.strftime('%Y-%m-%dT%H:%M')

def scrape(a):
    sDT = _normalize_dt(a['startDT'])
    eDT = _normalize_dt(a['endDT'])

    params = {
        'format': 'json',
        'sites': a['sites'],
        'startDT': sDT,
        'endDT': eDT,
        'parameterCd': a.get('parameterCd', '00060')
    }

    url = 'https://waterservices.usgs.gov/nwis/iv/'
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    raw = r.json()

    ts = raw['value']['timeSeries'][0]
    vb = ts['values'][0]['value']
    dte = pd.to_datetime([v['dateTime'] for v in vb], utc=True)
    vals = np.array([float(v['value']) for v in vb])

    return {
        'dte': dte,
        'data': vals,
        'siteName': ts['sourceInfo']['siteName'],
        'siteID': ts['sourceInfo']['siteCode'][0]['value']
    }

def interpolate_series(series, time_grid):
    return series.reindex(time_grid).interpolate(method='time')

def plot_sink_rise(sink, rise, freq='15min'):
    sink_s = pd.Series(sink['data'], index=sink['dte'])
    rise_s = pd.Series(rise['data'], index=rise['dte'])

    start = max(sink_s.index.min(), rise_s.index.min())
    end = min(sink_s.index.max(), rise_s.index.max())
    time_grid = pd.date_range(start=start, end=end, freq=freq)

    sink_i = interpolate_series(sink_s, time_grid)
    rise_i = interpolate_series(rise_s, time_grid)
    diff_i = sink_i - rise_i

    plt.figure(figsize=(12, 8))

    plt.subplot(2, 1, 1)
    plt.plot(time_grid, sink_i, 'k-', lw=2, label='sink')
    plt.plot(time_grid, rise_i, 'b--', lw=2, label='rise')
    plt.ylabel('Flow (cfs)')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(time_grid, diff_i, 'k-', lw=2, label='sink-rise')
    plt.axhline(0, color='r', ls='--')
    plt.xlabel('Datetime')
    plt.ylabel('sink-rise (cfs)')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
site_sink = widgets.Text(value='02321898', description='Sink Site:')
site_rise = widgets.Text(value='02321958', description='Rise Site:')
start_date = widgets.Text(value='2025-01-01 00:00', description='Start:')
end_date = widgets.Text(value='2026-06-30 00:00', description='End:')
freq_input = widgets.Text(value='15min', description='Freq:')
run_button = widgets.Button(description='Run')
output = widgets.Output()

def on_run_clicked(b):
    output.clear_output()
    with output:
        print('Fetching USGS data...')

        sink = scrape({
            'sites': site_sink.value,
            'startDT': start_date.value,
            'endDT': end_date.value
        })

        rise = scrape({
            'sites': site_rise.value,
            'startDT': start_date.value,
            'endDT': end_date.value
        })

        print('Plotting...')
        plot_sink_rise(sink, rise, freq=freq_input.value)

run_button.on_click(on_run_clicked)

display(widgets.VBox([
    start_date,
    end_date,
    run_button,
    output
]))